In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import pandas as pd
# en_data=pd.read_csv("/content/drive/MyDrive/Hope_ENG_test.csv", sep='\t')
# len(en_data)

2842

In [ ]:
# import pandas as pd
# ml_data=pd.read_csv("/content/drive/MyDrive/Hope_MAL_test.csv", sep='\t')
# len(ml_data)

1070

In [ ]:
# tal_data=pd.read_csv("/content/drive/MyDrive/Hope_KAN_test.csv", sep='\t')
# len(tal_data)

617

In [ ]:
1002,1070,1760

In [ ]:
# to convert emojis to text
!pip install emot
!pip install xgboost
!pip install Textblob

In [4]:
import numpy as np 
import pandas as pd
import re
import gc
import os
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
# import libraries and packages

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.feature_extraction.text import  CountVectorizer, TfidfTransformer,TfidfVectorizer
from sklearn.ensemble import VotingClassifier 
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV 

from textblob import TextBlob

import nltk.data
import numpy as np 
import pandas as pd
import re
import time
import string

pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
!pip install emoji

import emoji

def remove_emoji(text):
    return emoji.get_emoji_regexp().sub(u'', text)

     |████████████████████████████████| 175 kB 5.1 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5cfc3c1b792e48a93fb5c8061d0bf4967d39ff9cf547bbff8cfd7857a290c570
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [3]:
data= pd.read_csv("/content/drive/MyDrive/Hope_ENG_train.csv",header=None)
data.columns =['text','label']
data.head()


,text,label
0,these tiktoks radiate gay chaotic energy and i...,Non_hope_speech
1,@Champions Again He got killed for using false...,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Bla...,Non_hope_speech
4,Whenever we say black isn't that racists? Why...,Non_hope_speech


In [ ]:
len(data)

22740

In [ ]:
# for i in data['text']:
#   print(len(i))

In [5]:
import string
import re

def clean_text(text):
    text = ' '.join([word for word in text.split() if not word.startswith('@')])           #user name with @ is removed
    text = re.sub('@[a-zA-Z0-9]*', '', text)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) 
    text = "".join([word for word in text if not word.isdigit()])
    tokens = re.split('\W+',text)
    text = " ".join(wn.lemmatize(word) for word in tokens if word not in stopwords)
    return text



data_train= pd.read_csv("/content/drive/MyDrive/Hope_ENG_train.csv",header=None) # fix dataset path
data_dev= pd.read_csv("/content/drive/MyDrive/Hope_ENG_dev.csv", header=None)
#data_test= pd.read_csv("/content/drive/MyDrive/commaaticon_test_data (1)/test_hin.tsv", sep='\t')
data_train.columns=['Text','label']
data_dev.columns=['Text','label']
data_train.head()

,Text,label
0,these tiktoks radiate gay chaotic energy and i love it,Non_hope_speech
1,@Champions Again He got killed for using false money,Non_hope_speech
2,It's not that all lives don't matter,Non_hope_speech
3,Is it really that difficult to understand? Black lives matter and all lives matter are not mutually exclusive. They are both true as general statements. So are white lives matter and asian lives matter. Black lives m...,Non_hope_speech
4,Whenever we say black isn't that racists? Why don't just say Americans.,Non_hope_speech


In [ ]:
len(data_dev)

389

In [6]:
from imblearn.over_sampling import SMOTE
# Resampling the minority class. The strategy can be changed as required.
sm = SMOTE(sampling_strategy='minority', random_state=42)
# Fit the model to generate the data.


In [7]:
data_train['label'].value_counts()
  

Non_hope_speech    20778
Hope_speech         1962
Name: label, dtype: int64

In [8]:
data_dev['label'].value_counts()

Non_hope_speech    2569
Hope_speech         272
Name: label, dtype: int64

In [9]:
data_train['clean'] = data_train['Text'].apply(lambda x:clean_text(x))      
data_train.head(10)

data_dev['clean'] = data_dev['Text'].apply(lambda x:clean_text(x))      
data_dev.head(10)

,Text,label,clean
0,Thats it.... like....I dont like that statue,Non_hope_speech,thats likei dont like statue
1,@Generation X Counting money that she been given for promoting hysterical green agenda.,Non_hope_speech,x counting money given promoting hysterical green agenda
2,@Paola Hernandez i never said to be intolerant and hateful........ -_-,Non_hope_speech,hernandez never said intolerant hateful
3,@Firstlast300 Wow An opinion is that I don't like a color,Non_hope_speech,wow opinion dont like color
4,WOW!!!!!!!That was so so inspiring and incredible!!! THis speech has just saved my day,Hope_speech,wowthat inspiring incredible speech saved day
5,@FALC0n Yea sorry I know Asian is an ethnicity not a race... I'm just trying to generalize🤦‍,Non_hope_speech,yea sorry know asian ethnicity race im trying generalize
6,Blm nooooooonnAsian racism,Non_hope_speech,blm nooooooonnasian racism
7,What is making you think they are trying to make black lives matter more. I’m confused,Non_hope_speech,making think trying make black life matter confused
8,I dont agree with homosexualitynFirst of all you are homosexuality,Non_hope_speech,dont agree homosexualitynfirst homosexuality
9,@corinthia Long sorry not to fight about this,Non_hope_speech,long sorry fight


In [10]:
len(data_train['clean'])

22740

In [11]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class 
df_majority = data_train[(data_train['label']=='Non_hope_speech')] 
df_minority = data_train[(data_train['label']=='Hope_speech')] 
# upsample minority class
df_minority_upsampled = resample(df_minority,replace=True, n_samples= 20778, random_state=None)  # reproducible results
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled, df_majority])

In [12]:
df_upsampled['label'].value_counts()

Hope_speech        20778
Non_hope_speech    20778
Name: label, dtype: int64

In [13]:
#y_train=df_upsampled['label']
y_train=df_upsampled['label']
#y_test=data_dev['label']


y = pd.get_dummies(df_upsampled['label']).values


y_train1=pd.get_dummies(y_train).values
#y_test1=pd.get_dummies(y_test).values
diz_label = {}
for i,label in enumerate(df_upsampled.label.factorize()[1]):
    diz_label[i] = label
    
diz_label

{0: 'Hope_speech', 1: 'Non_hope_speech'}

In [14]:
df_upsampled.head()

,Text,label,clean
4436,Some of the black lives matter people are doing things correctly to protect their people but there are some that are destructive against innocent people and bad people too.When someone says black lives matter it is r...,Hope_speech,black life matter people thing correctly protect people destructive innocent people bad people toowhen someone say black life matter rude say life matter purpose movement black protection people rude bring life erase...
5727,even if shes talkin for women i as a male was really motivated by her. I always wanted to be an engineer but i lack of confidence that maybe im not smart enough(or not born smart enough) but its more like how bad you...,Hope_speech,even shes talkin woman male really motivated always wanted engineer lack confidence maybe im smart enoughor born smart enough like bad want
22681,Racism is everywhere.Though Many of the BLM people are doing some retarded shit. You wouldnt understand how they felt unless you were black.,Hope_speech,racism everywherethough many blm people retarded shit wouldnt understand felt unless black
3642,She had an engineering job. If you're so interested in her accomplishments look them up.,Hope_speech,engineering job youre interested accomplishment look
21822,All life's matters,Hope_speech,life matter


In [15]:
X_train=df_upsampled['clean'].tolist()
X_test =data_dev['clean'].tolist()




In [ ]:
len(X_train)

41556

In [ ]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
X_train1 = pad_sequences(sequences, maxlen=50)

sequences = tokenizer.texts_to_sequences(X_test)
X_test1 = pad_sequences(sequences, maxlen=50)



In [ ]:
type(X_test1)

numpy.ndarray

In [ ]:
model = Sequential()
model.add(Embedding(20000, 1000, input_length=50))
model.add(Dropout(0.2))
model.add(Conv1D(64, 2, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(2, activation='softmax'))
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 50, 1000)          20000000  
                                                                 
 dropout_2 (Dropout)         (None, 50, 1000)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 49, 64)            128064    
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 24, 64)           0         
 1D)                                                             
                                                                 
 lstm_2 (LSTM)               (None, 100)               66000     
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                      

The implementwd 1D CNN-LSTM model is composed of an input layer equal to 50, 64  convolutional layers, two  pooling layer and relu activation function used for conv1d layer, two LSTM layers, 100 fully connected (FC) layers, and a soft-max output layer.


In [ ]:
from sklearn.utils import class_weight

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

model.fit(X_train1, y_train1,
                    batch_size=254,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,)

Epoch 1/10
148/148 [==============================] - 27s 77ms/step - loss: 0.3616 - accuracy: 0.8362 - val_loss: 0.2128 - val_accuracy: 0.9148
Epoch 2/10
148/148 [==============================] - 9s 61ms/step - loss: 0.1357 - accuracy: 0.9529 - val_loss: 0.2309 - val_accuracy: 0.9237
Epoch 3/10
148/148 [==============================] - 8s 53ms/step - loss: 0.0818 - accuracy: 0.9729 - val_loss: 0.1911 - val_accuracy: 0.9398
Epoch 4/10
148/148 [==============================] - 7s 51ms/step - loss: 0.0580 - accuracy: 0.9815 - val_loss: 0.2650 - val_accuracy: 0.9264
Epoch 5/10
148/148 [==============================] - 8s 51ms/step - loss: 0.0474 - accuracy: 0.9838 - val_loss: 0.2333 - val_accuracy: 0.9343
Epoch 6/10
148/148 [==============================] - 8s 51ms/step - loss: 0.0430 - accuracy: 0.9857 - val_loss: 0.2230 - val_accuracy: 0.9451
Epoch 7/10
148/148 [==============================] - 8s 51ms/step - loss: 0.0405 - accuracy: 0.9861 - val_loss: 0.2398 - val_accuracy: 0.938

In [ ]:
preds = model.predict(X_test1)

In [ ]:
pred=np.argmax(preds,axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

label = le.fit(df_upsampled["label"])
label_t=le.fit_transform(df_upsampled["label"])
prediction2=le.inverse_transform(pred)

In [ ]:
score = model.evaluate(X_test1, y_test1,
                       batch_size=1000, verbose=1)
print('Test accuracy:', score[1])



3/3 [==============================] - 1s 397ms/step - loss: 1.1033 - accuracy: 0.8824
Test accuracy: 0.8824357390403748


In [ ]:

#sampled data
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(preds,axis=1)))

              precision    recall  f1-score   support

           0       0.34      0.24      0.28       272
           1       0.92      0.95      0.94      2569

    accuracy                           0.88      2841
   macro avg       0.63      0.60      0.61      2841
weighted avg       0.87      0.88      0.87      2841



In [ ]:
prediction2

In [ ]:
y_pred_df = pd.DataFrame(data=prediction2, columns=['prediction2'])
Kannada = pd.DataFrame()
Kannada['Text'] = data_dev['Text']
Kannada['label'] = prediction2
Kannada.to_csv('MUCIC_English.tsv',sep='\t',index = None)
Kannada.head()

,Text,label
0,These Abandon Hope videos only cement my pessimism towards humanity. Ugh,Non_hope_speech
1,Wow your videos are long. They have good points,Non_hope_speech
2,I know this is none of my business but 75k+ income?! Is your household only comprised of you? If so,Non_hope_speech
3,Hope? Is that a new Pokémon?,Non_hope_speech
4,Almost every time someone depicts a davidian star they make it blue,Non_hope_speech


Experiments down

In [ ]:
len(data_dev)

In [ ]:
print(classification_report(np.argmax(y_test1,axis=1),np.argmax(preds,axis=1)))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       272
           1       0.90      1.00      0.95      2569

    accuracy                           0.90      2841
   macro avg       0.45      0.50      0.47      2841
weighted avg       0.82      0.90      0.86      2841



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
